In [1]:
from Direct_Fidelity_Estimation_gates import *

In [105]:
θ = np.random.randn(1)
ϕ = 4*np.pi/5

In [106]:
U = np.array( [ [ np.exp(-1j*ϕ/2), 0 ] , [0, np.exp(1j*ϕ/2)] ] )
V = np.array( [ [ np.cos(θ), np.sin(θ)], [np.sin(θ), -np.cos(θ)] ] )

In [107]:
U

array([[0.30901699-0.95105652j, 0.        +0.j        ],
       [0.        +0.j        , 0.30901699+0.95105652j]])

In [108]:
V

array([[[-0.33864604],
        [ 0.94091384]],

       [[ 0.94091384],
        [ 0.33864604]]])

In [109]:
abs(np.trace(U.T.conj()@V))**2/4

array([0.10373007])

In [110]:
M = 5

In [111]:
σ = np.outer( V.flatten(), V.flatten().conj() )
n = int( np.log2( σ.shape[0] ) )
paulis = np.array( [ [1,0,0,1], [0,1,1,0], [0,-1j,1j,0], [1,0,0,-1] ] ).reshape(4,2,2) / np.sqrt(2) 
Pk = n*[ paulis ]
σk = np.real( InnerProductMatrices( σ, Pk ) ).flatten()
qk = np.real( σk**2 )
Index = rm.choices( range(4**n), qk, k = M )

In [112]:
σ

array([[ 0.11468114, -0.31863675, -0.31863675, -0.11468114],
       [-0.31863675,  0.88531886,  0.88531886,  0.31863675],
       [-0.31863675,  0.88531886,  0.88531886,  0.31863675],
       [-0.11468114,  0.31863675,  0.31863675,  0.11468114]])

In [113]:
ρ   = np.outer( U.flatten(), U.flatten().conj() )
ρk  = np.real( InnerProductMatrices( ρ, Pk ) ).flatten()
ρki = ρk[Index] 
ρki_exact = ρki

In [114]:
ρki

array([ 0.        ,  1.        ,  1.        ,  0.80901699, -0.80901699])

In [115]:
ρ

array([[ 1.        +0.j        ,  0.        -0.j        ,
         0.        -0.j        , -0.80901699-0.58778525j],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [ 0.        +0.j        ,  0.        +0.j        ,
         0.        +0.j        ,  0.        +0.j        ],
       [-0.80901699+0.58778525j,  0.        +0.j        ,
         0.        +0.j        ,  1.        +0.j        ]])

In [116]:
σki = σk[Index] 
Fid = np.real( np.sum( ρki/σki )/M ) 
Fid

0.3518430038254289

In [117]:
circ_U = QuantumCircuit(1)
circ_U.rz(ϕ,0)

In [118]:
Index_b4 = [ convert_to_base( index, 4, n) for index in Index]
Labels = Index_b4
Labels

['31', '00', '00', '22', '11']

In [119]:
n = circ_U.num_qubits
M = len(Labels)

if not isinstance(Labels, list):
    Labels = [Labels]

EigenValues = [
            np.array([ 1.,  1. ]),
            np.array([ 1., -1. ]),
            np.array([ 1., -1. ]),
            np.array([ 1., -1. ])
            ] 

Circuits = []
Labels_out = []
ExpectedValues = M*[None]
Indx = []

for m in range(M):
    label = Labels[m]
    if label == ''.zfill(2*n):
        ExpectedValues[m] = 1  
    else:
        Indx.append(m)
        Labels_out.append(label)
        circuit_0 = QuantumCircuit(n,n)
        circuit_1 = QuantumCircuit(n,n)
        
        for k in range(n):
            idex = label[::-1][n+k]
            if idex == '3' or  idex == '0':
                
                circuit_0.barrier()
                circuit_0.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_0.barrier()
                
                circuit_1.x(k)
                circuit_1.barrier()
                circuit_1.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_1.barrier()
                
            elif idex == '1':
                circuit_0.h(k)
                circuit_0.barrier()
                circuit_0.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_0.barrier()
                
                circuit_1.x(k)
                circuit_1.h(k)
                circuit_1.barrier()
                circuit_1.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_1.barrier()
                
            elif idex == '2':
                circuit_0.u2( np.pi/2, np.pi, k )
                circuit_0.barrier()
                circuit_0.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_0.barrier()
                                
                circuit_1.x(k)
                circuit_1.u2( np.pi/2, np.pi, k )
                circuit_1.barrier()
                circuit_1.compose( circ_U, qubits=range(n), inplace=True) 
                circuit_1.barrier()
                
        for k in range(n):
            idex = label[::-1][k]
            if idex == '1':
                circuit_0.h(k)
                circuit_1.h(k)
            elif idex == '2':
                circuit_0.u2( 0, -np.pi/2, k )
                circuit_1.u2( 0, -np.pi/2, k )
                
        circuit_0.measure( range(n), range(n) )
        circuit_1.measure( range(n), range(n) )
        
        Circuits.append( circuit_0 )
        Circuits.append( circuit_1 )

In [120]:
Circuits[0].draw()

░ ┌──────────┐ ░ ┌───┐┌─┐
q_0: ─░─┤ Rz(4π/5) ├─░─┤ H ├┤M├
      ░ └──────────┘ ░ └───┘└╥┘
c: 1/════════════════════════╩═
                             0

In [121]:
Circuits[1].draw()

┌───┐ ░ ┌──────────┐ ░ ┌───┐┌─┐
q_0: ┤ X ├─░─┤ Rz(4π/5) ├─░─┤ H ├┤M├
     └───┘ ░ └──────────┘ ░ └───┘└╥┘
c: 1/═════════════════════════════╩═
                                  0

In [122]:
len( Circuits )

6

In [123]:
quantum_instance = QuantumInstance( Aer.get_backend('aer_simulator'), shots=2**13 )

In [124]:
Job = quantum_instance.execute( Circuits )

In [140]:
probs = get_probabilities( Job )

In [133]:
probs.shape

(4, 3)

In [134]:
Labels_out

['31', '22', '11']

In [135]:
for j in range( 0,len(Labels_out)):
    ExpectedValues[Indx[j]] = LocalProduct( probs[:,j].T , 
                                  [ EigenValues[int(Labels_out[j][k])] for k in range(n) ]  )[0] 

In [136]:
ρki = ExpectedValues

In [137]:
σki = σk[Index] 
Fid = np.real( np.sum( ρki/σki )/M ) 
Fid 

0.6176317850482426

In [138]:
l = 7
print( np.array( [ ρki, ρki_exact, Labels] ).T )

[['-0.0126953125' '0.0' '31']
 ['1' '0.9999999999999997' '00']
 ['1' '0.9999999999999997' '00']
 ['0.013916015625' '0.8090169943749471' '22']
 ['0.8125' '-0.8090169943749471' '11']]


In [141]:
probs

array([[0.49365234, 0.50695801, 0.90625   , 0.09411621, 0.09533691,
        0.91003418],
       [0.50634766, 0.49304199, 0.09375   , 0.90588379, 0.90466309,
        0.08996582]])

In [142]:
Circuits[0].draw()

░ ┌──────────┐ ░ ┌───┐┌─┐
q_0: ─░─┤ Rz(4π/5) ├─░─┤ H ├┤M├
      ░ └──────────┘ ░ └───┘└╥┘
c: 1/════════════════════════╩═
                             0